# House Prices Competition

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(rc={'figure.figsize':(22,8.27)})

### Load Data

In [ ]:
train = pd.read_csv('./data/train.csv', index_col='Id')
test = pd.read_csv('./data/test.csv', index_col='Id')

In [ ]:
train.head()

### First outliers

In [ ]:
train['GrLivArea'].max(), test['GrLivArea'].max()

In [ ]:
# find outliers
plt.scatter(train['GrLivArea'], train['SalePrice'])

In [ ]:
# filter for outliers
sale_price_outliers_mask = (train['GrLivArea'] > 4000)

# drop outliers from training set
train.drop(train.loc[sale_price_outliers_mask].index, axis=0, inplace=True)

In [ ]:
plt.scatter(train['GrLivArea'], train['SalePrice'])

### All data

In [ ]:
# all data
data = pd.concat([train, test])

In [ ]:
data.shape

### Drop columns with too many missing values

In [ ]:
data.isnull().sum(axis=0).head(10)

In [ ]:
sns.heatmap(data.isnull())

In [ ]:
# drop all columns that have more than 70% of NaNs
for col in data.columns:
    if data[col].isna().sum() > 0.7 * data.shape[0]:
        data.drop(col, axis=1, inplace=True)
        print(col)

In [ ]:
sns.heatmap(data.isnull())

In [ ]:
data.info()

### Drop columns with different categories in train and test sets

In [ ]:
cols_to_drop = []

for col in data.drop('SalePrice', axis=1).columns:
    if train[col].dtype not in ['float64', 'int64']:
        if (data.loc[data[col].notnull(), col].unique() != train.loc[train[col].notnull(), col].unique()).any():
            cols_to_drop.append(col)
            print(col)

# data.drop(cols_to_drop, axis=1, inplace=True)

### Fill NaNs 
<b>in categorical features</b><br>
<i> - with mode</i><br>
<i> - with "Missing" if the number of missing values is too high</i><br>
<b>in numeric features</b><br>
<i> - with median</i><br>
<i> - with -999 if the number of missing values is too high</i>

In [ ]:
# define columns with missing values
na_columns = data.drop('SalePrice', axis=1).columns[(data.drop('SalePrice', axis=1).isna().sum() > 0)]

In [ ]:
# fill NaNs
for col in na_columns:
    if data[col].isnull().sum() < 0.4 * data.shape[0]:
        if str(data[col].dtype) == 'float64':
            data[col].fillna(data[col].median(), inplace=True)
        else:
            data[col].fillna(data[col].mode(), inplace=True)
            data[col].replace({None: data[col].mode()[0]}, inplace=True)
    else:
        if str(data[col].dtype) == 'object':
            data[col].fillna('Missing', inplace=True)
            data[col].replace({None: 'Missing'}, inplace=True)
        else:
            data[col].fillna(-999, inplace=True)

In [ ]:
sns.heatmap(data.isnull())

In [ ]:
# list of the numbers of unique values of the non-integer categorical features
values_arr = []
for col in data.columns:
    if data[col].dtype not in ['float64', 'int64']:
        values_arr.append(len(data[col].unique()))
np.array(values_arr)

### Feature engineering

In [ ]:
# age of the house
data['HouseAge'] = data['YrSold'] - data['YearBuilt']

# years since the last remodeling
data['ModelAge'] = data['YrSold'] - data['YearRemodAdd']

# is remodeled or not
# remodeled_mask = (data['YearBuilt'] != data['YearRemodAdd'])
# data['Remodeled'] = np.zeros(data.shape[0])
# data.loc[remodeled_mask, 'Remodeled'] = 1

# drop previous columns
data.drop(['YearBuilt', 'YearRemodAdd'], axis=1, inplace=True)

### Encode categorical features

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
lb = LabelBinarizer()
for col in data.drop('SalePrice', axis=1).columns:
    if data[col].dtype not in ['float64', 'int64']:
        matrix = lb.fit_transform(data[col]).T
        unique = data[col].unique()
        # if columns is binary, LabelBinarizer returns 1d-array instead of 2d-matrix
        if matrix.shape[0] == 1:
            data[col] = matrix[0]
        else:
            for i in range(len(unique)):
                data[str(col + unique[i])] = matrix[i]
            # remove previous column
            data.drop(col, axis=1, inplace=True)

In [ ]:
for col in data.columns:
    if len(data[col].unique()) == 1:
        print(col)

In [ ]:
data.head()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.head()

### Only important columns

In [ ]:
# data = data[['OverallQual', 'TotalBsmtSF', 'CentralAir', '1stFlrSF', 'GrLivArea',
#        'FullBath', 'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'RH', 'NWAmes',
#        'WdShngl', 'Av', '2Types', 'ConLw', 'SalePrice']]

### Train-test split

In [ ]:
y_train = train['SalePrice']
data.drop(['SalePrice'], axis=1, inplace=True)

train = data.iloc[:train.shape[0]]
test = data.iloc[train.shape[0]:]

### Model Selection

In [ ]:
# import models
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# import instruments for model evaluation
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse

In [ ]:
# define error function [RMSE(log(y), log(y_pred))]
def error(Y_real, Y_pred):
    return np.sqrt(mse(np.log(Y_real), np.log(Y_pred)))

In [ ]:
# list of models
models = [
    XGBRegressor(),
    RandomForestRegressor()
]

# K-Fold splitter
kfold = KFold(n_splits=10)

# Model evaluation DataFrame
log_df = pd.DataFrame(columns=['Model', 'Error'])
log_dict = {}

for train_ind, test_ind in kfold.split(train, y_train):
    X_train, Y_train = train.iloc[train_ind], y_train.iloc[train_ind]
    X_test, Y_test = train.iloc[test_ind], y_train.iloc[test_ind]
    
    for model in models:
        name = model.__class__.__name__
        
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        
        err = error(Y_test, Y_pred)
        
        if name in log_dict:
            log_dict[name] += err
        else:
            log_dict[name] = err

for model in log_dict:
    row = {'Model': model, 'Error': log_dict[model] / 10}
    log_df = log_df.append(row, ignore_index=True)

log_df

### Prediction with the best model

In [ ]:
best = XGBRegressor()
best.fit(train, y_train)

predictions = best.predict(test)

In [ ]:
plt.plot(train.columns, best.feature_importances_)

In [ ]:
predictions

In [ ]:
submission_df = pd.read_csv('./data/sample_submission.csv', index_col='Id')
submission_df['SalePrice'] = predictions

In [ ]:
submission_df.to_csv('./result/XGB.csv')